In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=d38ad18646f47e33d1ea13884ae474e2c24aaddcaf52bb4a89bcedbb573ed90e
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [4]:
## Read The dataset
training = spark.read.csv('test1_data.csv',header=True,inferSchema=True)

In [5]:
training.show()


+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|     ali| 31|        10| 30000|
|    reza| 30|         8| 25000|
|   akbar| 29|         4| 20000|
|pedramoo| 24|         3| 20000|
| hasanoo| 21|         1| 15000|
|    kami| 23|         2| 18000|
+--------+---+----------+------+



In [6]:
training.printSchema()


root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns


['Name', 'age', 'Experience', 'Salary']

In [8]:
# [Age,Experience]----> new feature--->independent feature


In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [10]:
output=featureassembler.transform(training)


In [11]:
output.show()


+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|     ali| 31|        10| 30000|         [31.0,10.0]|
|    reza| 30|         8| 25000|          [30.0,8.0]|
|   akbar| 29|         4| 20000|          [29.0,4.0]|
|pedramoo| 24|         3| 20000|          [24.0,3.0]|
| hasanoo| 21|         1| 15000|          [21.0,1.0]|
|    kami| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [ ]:
output.columns


In [17]:
finalized_data=output.select("Independent Features","Salary")


In [18]:
finalized_data.show()


+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [20]:
### Coefficients
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [21]:
### Intercepts
regressor.intercept


22295.299605311826

In [22]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [23]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17203.085755292585|
+--------------------+------+------------------+



In [24]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError


(2203.0857552925845, 4853586.845173097)